Segmenting and Clustering in Toronto 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import requests # library to handle requests and web scraping

from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print("everything was installed")

everything was installed


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

print("The soup is hot and ready")

The soup is hot and ready


Webscraping and putting everything into a table and rows.

In [3]:
data_table = soup.find('table', {'class': 'wikitable sortable'})
data_rows = data_table.find_all('tr')

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
data=[]
for row in data_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    
df_toronto = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto = df_toronto[~df_toronto['PostalCode'].isnull()] # this filters out the bad rows.
df_toronto.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [5]:
df_toronto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode      287 non-null object
Borough         287 non-null object
Neighborhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [6]:
df_toronto.shape

(287, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df_toronto.drop(df_toronto[df_toronto['Borough'] =="Not assigned"].index, axis=0, inplace=True)
df_toronto

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
10,M9A,Queen's Park,Not assigned
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [8]:
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [9]:
df_tor_group = df_toronto.groupby('PostalCode').agg(lambda x: ','.join(x))
df_tor_group.reset_index(inplace=True)
df_tor_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df_tor_group.shape

(103, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [11]:
df_tor_group.loc[df_tor_group['Neighborhood'] == "Not Assigned", 'Neighborhood'] = df_tor_group.loc[df_tor_group['Neighborhood']=="Not assigned", 'Borough']
df_tor_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,"Scarborough,Scarborough","Rouge,Malvern"
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Remove the duplicate the boroughs.

In [12]:
df_tor_group['Borough']=df_tor_group['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_tor_group.set_index('PostalCode', inplace=True)
df_tor_group

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


In [13]:
df_tor_group.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [14]:
df_tor_group.shape

(103, 2)